# TextRank

### Recap Tf-Idf method
- split doc into sentences
- compute tfidf matrix (sentences x terms)
- compute scores as average of nonzero values
- take top scoring sentences

#### TextRank is an alternative way for scoring sentences, instead of taking the average for each sentence, it use a more complex method, all the other steps remain the same

## Google PageRank
- TextRank is based on Google's PageRank
- remember: internet is made of webpages each of which can potentially return as the search result
- we wanna compute a score for each webpage
- we can rank the search results according to their scores
### How do we score each webpage?
- through what is called random walk
- start from an arbitrary page, select a link on that webpage, go to that page, and repeat this process after doing some math, repeat forever...
- after the fancy map, magic happens: it turns out probability of landing on a webpage, after a certain amount of time remains constant
- it doesn't matter where you start from!!!
- no matter where you start if you keep going from webpage to webpage randomly, you'll eventually settle on some fixed distribution

- Intuitviely a webpage with more incoming links will be more popular and will have a higher chance of being landed on
- these probabilities are in fact page rank scores
- the assumption is that more authoritative and legitimate pages will have more incoming links
- what if people just make a bunch of website that link to their own website
- As an exercise think why that won't work
### Applying PageRank to TextRank (somewhat advanced)
- treat every sentence as a webpage
- what is the equivalent of incoming links?
- the number of links from one sentence to another is the cosine similarity between their tfidf vectors!
- Intuitviely if many sentences have high cosine similarity to one authoritative sentence, that authoritative sentence will end up in a higher score NOTE THIS IS NOT SYMMETRIC EVEN THOUGH THE COSINE SIMILARITY IS!
- because the probability of walking from one sentence to another will depend on how many other sentences it is similar to
- let's say authoritative sentence will have high cosine similarity to multiple other sentences, the probability of taking a random walk towards each will be much smaller, as the prob will spread among those sentences
- as with PageRank, the probability of landing on a particular sentence will eventually settle on a fixed distributon
- and these will be textrank scores

## How does it really work? Mechanics of PageRank / TextRank
- this is a really compressed version of what's going on, normalde buna ayrı bi ders grekir dedi
### Random Walks and Markov Chains
random walks is an example of Markov chains
- markov chains made up of a set of states
- we can go from any state at time t to another state in time t+1 according to some given probabilities
- suppose we have M states in total (in this case M webpages)
- the probability going from state i at time t, to state j in time t+1 : A(i,j) = p(st+1 = j| st=i)
- this transition follow markov property (only depend on whre i was previously but not on steps before tht)
- next step: how we represent the probability of being in a particular state?
#### This is called State Distribution for each state
- p(st) = [p(st =1), p(st=2)...p(st=M)]
- so p(st) is a vector of probabilities values telling us the prob of being at a particular state at time t.
- By convention, it's a 1xM row vector
#### The Next State Distribution:
- we can use the former with A matrix to compute the state distribution in the next step
- p(st+1) = p(st) * A #convince yourself.
- Mantıklı çünkü bir state'te olma prob'unu row vektör yaptık, e sonra A'nın kolonlarıyla çarpınca ikinci state'teki probability distribution çıkıyo
#### The Limiting Distribution: state distribution after an infinitely long random walk
- p(s0)*A*A*A... = lim p(s0) A^t :t sonsuza giderken
- no guarantees that it will converge, or it exists
- LUCKILY A TRICK TO GUARANTEE THIS DISTRIBUTION DOES EXIST AND FIND IT NOT IN INF NUMBER STEPS

- t= sonsuzdan başlayalım p(s-sonsuz) = p(s-sonsuz)* A .ünkü sonsuz + 1 yine sonsuz.
- this is a typical Eigenvalue problem: A*v = lambda*v 
- ps-sonsuz is an eigenvector, eigenvalue is 1.
- bu denklemin sağ tarafı is not limiting distribution but a stationary distribution (a distribution that doesn't change after 1 transition by A)
- question : how do we know that this stationary distribution is also a limiting distribution
- how do we know it's unique?
- how do we know the corresponding eigenvalue is 1, to begin with?
#### Answer: Perron-Frobenius Theorem
- if A is a Markov Matrix (stochastic matrix) and we can travel from any state to another state with positive probability, then all the assumptions above are true
- you don't need to prove this theory, just to check if criteria are met
1- A is a Markov Matrix (each rows sums to 1 and values are non-neg)

2- All values ARE positive (non-neg not enough)
- if 1 and 2 holds, A has an eigenvalue of 1, limiting dist is same as stationary dist and its unique

- Normally in a webpage i, if the number of links are N, going to each of these links randomly from page i will be 1/N(i)
- and of course the probability will be 0 for the pages which does not contained as link in page i
- let's build a matrix G from these probabilities
- since G contain zero values, it's not meeting Perron-Frobenius Theorem
- What we'll do is we'll smooth matrix G by a uniform matrix U which contains 1/M everywhere to obtain a matrix A which meets the criterion
- A = æ * U + (1-æ) * G 
- the sums will still be 1, and no zero values.

#### Let's move on TextRank
- we just computed tf-idf matrix
- next: compute the cosine similarity pairwise between each sentence
- This yields an MxM matrix that shows similarity between sentence i and sentence j
- we cannot use it as it is since rows does not sum to 1.
- divide each row by sum to enforce this constraint, now it's our G matrix
- smooth G to get A

## Exercise Prompt
- only use Numpy but not any library for the advanced application
- Hint: most code from previous notebook can be used
- sentence tokenization, tfidf, ranking by scoe are same
- conventional eigenvector representation is transpose of how it's presented in markov chains
p = p*A
- since we treat the state distribution as a row vector, this chage how we call eigenvalue function in np
A^T * p^T = p^T

### Solutions (Advanced)

In [1]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yagmuraslan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yagmuraslan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

File 'bbc_text_cls.csv' already there; not retrieving.



In [7]:
df = pd.read_csv('bbc_text_cls.csv')
doc = df[df.labels == 'business']['text'].sample(random_state=42)

In [8]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [9]:
print(wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [10]:
print(doc.iloc[0].split("\n", 1)[1])


UK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.

Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.

The last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.

The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. The November-December jump last year was roughl

In [11]:
sents = nltk.sent_tokenize(doc.iloc[0].split("\n", 1)[1])

featurizer = TfidfVectorizer(
    stop_words=stopwords.words('english'),
    norm='l1')

X = featurizer.fit_transform(sents)

#### buraya kadar Text Summarization notebook'un aynısıydı

In [13]:
# compute similarity matrix
S = cosine_similarity(X) #bu fonksiyonu biliyoduk ama burda direkt matrikse uyguladık, a double for loop yapıyo
S.shape

(17, 17)

In [14]:
len(sents)

17

In [15]:
# normalize similarity matrix
S /= S.sum(axis=1, keepdims=True)

In [16]:
S[0].sum() #prob'ar toplamı 1 ediyo

1.0

In [17]:
# uniform transition matrix
U = np.ones_like(S) / len(S)

In [18]:
U[0].sum() #bunun da prob'lar toplamı 1

1.0

In [19]:
# smoothed similarity matrix
factor = 0.15
S = (1 - factor) * S + factor * U
S[0].sum() #haliyle bunun da

1.0

In [20]:
# find the limiting / stationary distribution
#he'll show both eigenvalue and eigenvector since both work
eigenvals, eigenvecs = np.linalg.eig(S.T) #we need to transpose our matrix because numpy implements the eig function
#to work on column vectors by default, therefore we're transposing it to work on a row vector
#6.dk da açıkılıyo

In [21]:
eigenvals #recall one of these eigenvalues should be one

array([1.        , 0.24245466, 0.72108199, 0.67644122, 0.34790129,
       0.34417302, 0.3866884 , 0.40333562, 0.41608572, 0.44238593,
       0.63909999, 0.62556792, 0.58922572, 0.57452382, 0.48511399,
       0.51329157, 0.52975372])

In [22]:
eigenvecs[:,0] #next step is to check the corrsponding eigenvector
#probably not good to assume that the corresponding eigenvector to eigenvalue 1 will be found in position 0.
#better to index explicitly
# we need to index the first column bcz eigenvectors goes by columns.

array([-0.24206557, -0.27051337, -0.2213806 , -0.28613638, -0.25065894,
       -0.2499217 , -0.279622  , -0.21515455, -0.2226665 , -0.22745415,
       -0.2059112 , -0.20959727, -0.23526242, -0.24203809, -0.23663025,
       -0.2940483 , -0.20865607])

In [23]:
eigenvecs[:,0].dot(S) #sanity check: bakalım bu eigenvector yü matrix'le çarpınca gene aynı eigenvector mü gelcek

array([-0.24206557, -0.27051337, -0.2213806 , -0.28613638, -0.25065894,
       -0.2499217 , -0.279622  , -0.21515455, -0.2226665 , -0.22745415,
       -0.2059112 , -0.20959727, -0.23526242, -0.24203809, -0.23663025,
       -0.2940483 , -0.20865607])

In [24]:
#size bu eigenvector yanlış gelmiş olabilir, değerler negatif, ama unutma eigenvectorler tek değil ve 
# herhangi bi sayıyla çarpılabilir (öyle mi?) 
#new eigenvector will still be parallel with the original
#bu şekilde 0-1 arası ve toplamı 1 eden pozitif olmasını sağlıycaz
eigenvecs[:,0] / eigenvecs[:,0].sum()

array([0.05907327, 0.06601563, 0.05402535, 0.06982824, 0.06117038,
       0.06099047, 0.06823848, 0.05250595, 0.05433915, 0.05550753,
       0.05025022, 0.05114976, 0.05741304, 0.05906657, 0.05774684,
       0.07175905, 0.05092007])

#### Now here is a simpler way to find the same limiting distribution

In [25]:
limiting_dist = np.ones(len(S)) / len(S) #to initiliza to be uniform dist
threshold = 1e-8
delta = float('inf')
iters = 0
while delta > threshold:
  iters += 1

  # Markov transition
  p = limiting_dist.dot(S)

  # compute change in limiting distribution
  delta = np.abs(p - limiting_dist).sum()

  # update limiting distribution
  limiting_dist = p

print(iters)

#it only took 41 steps? 
#to convergee??  anlamadım bu kodu hi.

41


In [26]:
limiting_dist

array([0.05907327, 0.06601563, 0.05402534, 0.06982824, 0.06117038,
       0.06099047, 0.06823848, 0.05250595, 0.05433915, 0.05550753,
       0.05025022, 0.05114977, 0.05741304, 0.05906657, 0.05774685,
       0.07175905, 0.05092008])

In [27]:
np.abs(eigenvecs[:,0] / eigenvecs[:,0].sum() - limiting_dist).sum() #logical this is where we set the threshold

1.9964739014777244e-08

In [28]:
scores = limiting_dist

In [29]:
sort_idx = np.argsort(-scores)

In [30]:
print("Generated summary:")
for i in sort_idx[:5]:
  print(wrap("%.2f: %s" % (scores[i], sents[i])))

Generated summary:
0.07: "The retail sales figures are very weak, but as Bank of England
governor Mervyn King indicated last night, you don't really get an
accurate impression of Christmas trading until about Easter," said Mr
Shaw.
0.07: A number of retailers have already reported poor figures for
December.
0.07: The ONS echoed an earlier caution from Bank of England governor
Mervyn King not to read too much into the poor December figures.
0.07: Retail sales dropped by 1% on the month in December, after a
0.6% rise in November, the Office for National Statistics (ONS) said.
0.06: Clothing retailers and non-specialist stores were the worst hit
with only internet retailers showing any significant growth, according
to the ONS.


In [31]:
def summarize(text, factor = 0.15):
  # extract sentences
  sents = nltk.sent_tokenize(text)

  # perform tf-idf
  featurizer = TfidfVectorizer(
      stop_words=stopwords.words('english'),
      norm='l1')
  X = featurizer.fit_transform(sents)

  # compute similarity matrix
  S = cosine_similarity(X)

  # normalize similarity matrix
  S /= S.sum(axis=1, keepdims=True)

  # uniform transition matrix
  U = np.ones_like(S) / len(S)

  # smoothed similarity matrix
  S = (1 - factor) * S + factor * U

  # find the limiting / stationary distribution
  eigenvals, eigenvecs = np.linalg.eig(S.T)

  # compute scores
  scores = eigenvecs[:,0] / eigenvecs[:,0].sum()
  
  # sort the scores
  sort_idx = np.argsort(-scores)

  # print summary
  for i in sort_idx[:5]:
    print(wrap("%.2f: %s" % (scores[i], sents[i])))

In [32]:
doc = df[df.labels == 'entertainment']['text'].sample(random_state=123)
summarize(doc.iloc[0].split("\n", 1)[1])

0.11: Goodrem, Green Day and the Black Eyed Peas took home two awards
each.
0.10: As well as best female, Goodrem also took home the Pepsi Viewers
Choice Award, whilst Green Day bagged the prize for best rock video
for American Idiot.
0.10: Other winners included Green Day, voted best group, and the
Black Eyed Peas.
0.10: The Black Eyed Peas won awards for best R 'n' B video and
sexiest video, both for Hey Mama.
0.10: Local singer and songwriter Missy Higgins took the title of
breakthrough artist of the year, with Australian Idol winner Guy
Sebastian taking the honours for best pop video.


In [33]:
doc.iloc[0].split("\n")[0]

'Goodrem wins top female MTV prize'

### Solution : Beginner - with a library

In [34]:
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 27.9 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21695 sha256=8c91a7dd304be24d347e6f65f6e5740c731c3ae113c9339a4ac864f99f06e340
  Stored in directory: /Users/yagmuraslan/Library/Caches/pip/wheels/ba/9f/70/7795228568b81b57a8932755938da9fb1f291b0576752604aa
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=54d4e31d3cb9ce1f9ca5184da377258fe74e35d70bef02319bfc494e3839ecdb
  Stored in directory: /Users/yagmuraslan/Library/Caches/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
  Created wheel for pycountry: filen

In [35]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [37]:
summarizer = TextRankSummarizer() #this doesn't take text directly as nlp can work in many languages

#the next function will take a text parser object and a tokenizer object:
parser = PlaintextParser.from_string(
    doc.iloc[0].split("\n", 1)[1],
    Tokenizer("english"))

summary = summarizer(parser.document, sentences_count=5) #call summirizer parse the doc and tell how many sentences return
summary
#this lib also has different methods to decide which sentence to keep top percent etc

(<Sentence: The 21-year-old singer won the award for best female artist, with Australian Idol runner-up Shannon Noll taking the title of best male at the ceremony.>,
 <Sentence: As well as best female, Goodrem also took home the Pepsi Viewers Choice Award, whilst Green Day bagged the prize for best rock video for American Idiot.>,
 <Sentence: The Black Eyed Peas won awards for best R 'n' B video and sexiest video, both for Hey Mama.>,
 <Sentence: Local singer and songwriter Missy Higgins took the title of breakthrough artist of the year, with Australian Idol winner Guy Sebastian taking the honours for best pop video.>,
 <Sentence: The ceremony was held at the Luna Park fairground in Sydney Harbour and was hosted by the Osbourne family.>)

In [38]:
for s in summary:
    print(wrap(str(s)))

The 21-year-old singer won the award for best female artist, with
Australian Idol runner-up Shannon Noll taking the title of best male
at the ceremony.
As well as best female, Goodrem also took home the Pepsi Viewers
Choice Award, whilst Green Day bagged the prize for best rock video
for American Idiot.
The Black Eyed Peas won awards for best R 'n' B video and sexiest
video, both for Hey Mama.
Local singer and songwriter Missy Higgins took the title of
breakthrough artist of the year, with Australian Idol winner Guy
Sebastian taking the honours for best pop video.
The ceremony was held at the Luna Park fairground in Sydney Harbour
and was hosted by the Osbourne family.


In [39]:
summarizer = LsaSummarizer() ### latent semantic analysis summarizer, also included in sumie package
#all the summarizer in this library has the same API so he won't explain syntax again
summary = summarizer(parser.document, sentences_count=5)
for s in summary:
    print(wrap(str(s)))

Goodrem, known in both Britain and Australia for her role as Nina
Tucker in TV soap Neighbours, also performed a duet with boyfriend
Brian McFadden.
Other winners included Green Day, voted best group, and the Black Eyed
Peas.
Goodrem, Green Day and the Black Eyed Peas took home two awards each.
As well as best female, Goodrem also took home the Pepsi Viewers
Choice Award, whilst Green Day bagged the prize for best rock video
for American Idiot.
Artists including Carmen Electra, Missy Higgins, Kelly Osbourne, Green
Day, Ja Rule and Natalie Imbruglia gave live performances at the
event.


In [41]:
#Imstead of dealing with parsers and summarizers, gensim has a method that you can directly pass the test
#It also uses TextRank
#it happens to use a variation of similarity fnction

# https://radimrehurek.com/gensim_3.8.3/summarization/summariser.html (documentation)
# https://arxiv.org/abs/1602.03606

# Parameters
# text (str) – Given text.
# ratio (float, optional) – Number between 0 and 1 that determines the
#     proportion of the number of sentences of the original text to be
#     chosen for the summary.
# word_count (int or None, optional) – Determines how many words will the
#     output contain. If both parameters are provided, the ratio will be
#     ignored.
# split (bool, optional) – If True, list of sentences will be returned.
#     Otherwise joined strings will bwe returned.
import gensim
from gensim.summarization.summarizer import summarize
summary = summarize(doc.iloc[0].split("\n", 1)[1])
print(wrap(summary))

ModuleNotFoundError: No module named 'gensim.summarization'

they’ve said that the gensim.summarization module has been removed in versions Gensim 4.x because it was an unmaintained third-party module.

To continue using gensim.summarization, you will have to downgrade the version of Gensim in requirements.txt. Try replacing it with gensim==3.8.3 or older.